### 1. read the train data and call basic cleaning function


In [27]:
#import data from a csv file and store in a dataframe with only 4 columns we need
import pandas as pd
df = pd.read_csv('../data/Hotel_Reviews.csv')

review_df = df[['Hotel_Name', 'Review_Date', 'Positive_Review', 'Negative_Review']]
review_df


,Hotel_Name,Review_Date,Positive_Review,Negative_Review
0,Hotel Arena,8/3/2017,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...
1,Hotel Arena,8/3/2017,No real complaints the hotel was great great ...,No Negative
2,Hotel Arena,7/31/2017,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...
3,Hotel Arena,7/31/2017,Great location in nice surroundings the bar a...,My room was dirty and I was afraid to walk ba...
4,Hotel Arena,7/24/2017,Amazing location and building Romantic setting,You When I booked with your company on line y...
...,...,...,...,...
515733,Atlantis Hotel Vienna,8/30/2015,location,no trolly or staff to help you take the lugga...
515734,Atlantis Hotel Vienna,8/22/2015,Breakfast was ok and we got earlier check in,The hotel looks like 3 but surely not 4
515735,Atlantis Hotel Vienna,8/19/2015,No Positive,The ac was useless It was a hot week in vienn...
515736,Atlantis Hotel Vienna,8/17/2015,The rooms are enormous and really comfortable...,No Negative


In [28]:

# Positive_Review: invalid reviews are with the following strings:'There are no comments available for this review', 'everything'
train_pos_invalid_content = 'nothing|everything|no positive'
# Negative_Review: invalid reviews are with the following strings:
train_neg_invalid_content = 'nothing|everything|anything|no negative'


In [29]:

import sys

# Add the directory containing the module to the Python path
sys.path.append('/Users/zengsheng/code/TechLah/RevuSum')

# Import the module that contains the function
import app.basic_review_clean as brc



In [30]:
# get the reviews for one hotel
review_df_now = review_df[review_df['Hotel_Name'] == 'Hotel Arena'] 
review_df_now

,Hotel_Name,Review_Date,Positive_Review,Negative_Review
0,Hotel Arena,8/3/2017,Only the park outside of the hotel was beauti...,I am so angry that i made this post available...
1,Hotel Arena,8/3/2017,No real complaints the hotel was great great ...,No Negative
2,Hotel Arena,7/31/2017,Location was good and staff were ok It is cut...,Rooms are nice but for elderly a bit difficul...
3,Hotel Arena,7/31/2017,Great location in nice surroundings the bar a...,My room was dirty and I was afraid to walk ba...
4,Hotel Arena,7/24/2017,Amazing location and building Romantic setting,You When I booked with your company on line y...
...,...,...,...,...
400,Hotel Arena,8/17/2015,Lovely location helpful staff The bikes avail...,Expensive to get to from the train station ai...
401,Hotel Arena,8/16/2015,EVERYTHING,Doors to the bathroom
402,Hotel Arena,8/9/2015,Room was bright and airy and staff were most ...,No Negative
403,Hotel Arena,8/4/2015,The staff were so friendly and very helpful,No Negative


In [31]:
## test the function ###
cleaned_review_df = brc.clean_hotel_reviews(review_df_now, train_pos_invalid_content, train_neg_invalid_content)
print(cleaned_review_df.head())

/Users/zengsheng/code/TechLah/RevuSum/app/basic_review_clean.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_reviews.loc[english_reviews[column_name].str.contains(invalid_content_str, case=False), column_name] = np.nan
/Users/zengsheng/code/TechLah/RevuSum/app/basic_review_clean.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_reviews.loc[english_reviews[column_name] == '', column_name] = np.nan


    Hotel_Name Review_Date                                    Positive_Review  \
0  Hotel Arena    8/3/2017   Only the park outside of the hotel was beauti...   
1  Hotel Arena    8/3/2017   No real complaints the hotel was great great ...   
2  Hotel Arena   7/31/2017   Location was good and staff were ok It is cut...   
3  Hotel Arena   7/31/2017   Great location in nice surroundings the bar a...   
4  Hotel Arena   7/24/2017    Amazing location and building Romantic setting    

                                     Negative_Review  
0   I am so angry that i made this post available...  
1                                                NaN  
2   Rooms are nice but for elderly a bit difficul...  
3   My room was dirty and I was afraid to walk ba...  
4   You When I booked with your company on line y...  


In [32]:
cleaned_review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 404
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Hotel_Name       357 non-null    object
 1   Review_Date      357 non-null    object
 2   Positive_Review  336 non-null    object
 3   Negative_Review  244 non-null    object
dtypes: object(4)
memory usage: 13.9+ KB


### 2. Preprocessing

In [33]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag


def preprocessing (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
  
    # tag each word with its part of speech
    tagged_words = pos_tag(tokenized)
    # remove adj and adv
    filtered_words = [word for word, tag in tagged_words if tag not in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']]

    words_only = [word for word in filtered_words if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword lists 
    #add more stop words
    stop_words.update(['hotel', 'booking', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten'])
    #print(len(stop_words))
    
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    cleaned = ' '.join(lemmatized) # Join back to a string
    # print(f'{lowercased=}\n', f'{tokenized=}\n', f'{filtered_words=}\n', f'{without_stopwords=}\n', f'{words_only=}\n', f'{lemmatized=}\n', f'{cleaned=}\n')
    # print()
    return cleaned

preprocessing(cleaned_review_df['Positive_Review'][0])

'park outside'

In [62]:

# preporcessing the positive and negative reviews and save them in two new columns:
cleaned_review_df['pos_processed_text'] = cleaned_review_df.Positive_Review.apply(lambda x: preprocessing(x) if not pd.isna(x) else x)
cleaned_review_df['neg_processed_text'] = cleaned_review_df.Negative_Review.apply(lambda x: preprocessing(x) if not pd.isna(x) else x)
df_now = cleaned_review_df.copy()

# merge the positive and negative reviews into one new column:'processed_text'
cleaned_review_df['processed_text'] = cleaned_review_df['pos_processed_text'] + cleaned_review_df['neg_processed_text']



#drop those rows with empty processed_text
df_now = cleaned_review_df[['processed_text']].dropna()
df_now


,processed_text
0,park outsidemade post via site use planing tri...
2,location staff breakfast range goroom bit room...
3,location surroundings bar restaurant area buil...
4,amazing location building settingbooked compan...
5,restaurant design chill place park nearby stai...
...,...
391,lot charm nice room bedcity tram ride
396,room bed decor charactershower didnt door floo...
397,room decor artwork around staffsituated town
399,staff location printer computer


In [64]:

#convert df series to list
doc = df_now['processed_text'].tolist()
# check the word count per item in the doc list




['park outsidemade post via site use planing trip make mistake place made via com stayed night july upon arrival placed room floor turned room booked reserved level room would window ceiling room mind broken window closed rain mini fridge contained sort bio weapon guessed smell asked change room explaining time booked btw cost got way volume ceiling offered room day check day clock order get room waned way begin holiday wait till order check room waist time room got wanted peaceful garden view window tired waiting room placed belonging rushed city evening turned noise room guess made vibrating tube something annoying hell stop making fall asleep wife audio recording attach want send via mail day came determine cause disturbing sound offered change room booked room left seems',
 'location staff breakfast range goroom bit room story step ask level inside room coffee boiler bar fridge',
 'location surroundings bar restaurant area building characterroom walk barefoot floor looked cleaned w

In [69]:
#check the dimension of the doc list
type(doc[1])

str

### 3. Keyword Extraction using KeyBERT

In [74]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

# function to get the keywords from the text using KeyBERT
def keywords_extract(text, top_n=5):
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
    kw_model = KeyBERT(model=sentence_model)
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), top_n=top_n)
    return keywords

keywords = keywords_extract(doc, 100)
keywords

[[('room', 0.4068),
  ('park', 0.3957),
  ('booked', 0.3467),
  ('garden', 0.3106),
  ('place', 0.304),
  ('reserved', 0.3001),
  ('mail', 0.2843),
  ('noise', 0.2803),
  ('sound', 0.2787),
  ('outsidemade', 0.2777),
  ('audio', 0.2738),
  ('site', 0.2713),
  ('recording', 0.2674),
  ('post', 0.2668),
  ('planing', 0.2652),
  ('trip', 0.2585),
  ('evening', 0.2519),
  ('closed', 0.2471),
  ('window', 0.2447),
  ('check', 0.243),
  ('contained', 0.2424),
  ('mind', 0.2408),
  ('disturbing', 0.2404),
  ('floor', 0.2371),
  ('change', 0.2325),
  ('holiday', 0.2298),
  ('arrival', 0.2283),
  ('sort', 0.226),
  ('ceiling', 0.2257),
  ('mistake', 0.2243),
  ('btw', 0.2202),
  ('wanted', 0.2186),
  ('belonging', 0.2154),
  ('night', 0.2154),
  ('order', 0.2144),
  ('stayed', 0.2053),
  ('fridge', 0.2049),
  ('placed', 0.202),
  ('got', 0.2007),
  ('asked', 0.1867),
  ('rain', 0.1853),
  ('city', 0.1849),
  ('com', 0.1846),
  ('clock', 0.1799),
  ('begin', 0.1744),
  ('broken', 0.1733),
  ('vo

In [75]:
len(keywords)

236

In [76]:
# flatten the list of list
one_list = [item for sublist in keywords for item in sublist]
one_list


[('room', 0.4068),
 ('park', 0.3957),
 ('booked', 0.3467),
 ('garden', 0.3106),
 ('place', 0.304),
 ('reserved', 0.3001),
 ('mail', 0.2843),
 ('noise', 0.2803),
 ('sound', 0.2787),
 ('outsidemade', 0.2777),
 ('audio', 0.2738),
 ('site', 0.2713),
 ('recording', 0.2674),
 ('post', 0.2668),
 ('planing', 0.2652),
 ('trip', 0.2585),
 ('evening', 0.2519),
 ('closed', 0.2471),
 ('window', 0.2447),
 ('check', 0.243),
 ('contained', 0.2424),
 ('mind', 0.2408),
 ('disturbing', 0.2404),
 ('floor', 0.2371),
 ('change', 0.2325),
 ('holiday', 0.2298),
 ('arrival', 0.2283),
 ('sort', 0.226),
 ('ceiling', 0.2257),
 ('mistake', 0.2243),
 ('btw', 0.2202),
 ('wanted', 0.2186),
 ('belonging', 0.2154),
 ('night', 0.2154),
 ('order', 0.2144),
 ('stayed', 0.2053),
 ('fridge', 0.2049),
 ('placed', 0.202),
 ('got', 0.2007),
 ('asked', 0.1867),
 ('rain', 0.1853),
 ('city', 0.1849),
 ('com', 0.1846),
 ('clock', 0.1799),
 ('begin', 0.1744),
 ('broken', 0.1733),
 ('volume', 0.1727),
 ('smell', 0.1713),
 ('view', 0

In [77]:
# convert the list to a DataFrame
keyword_df = pd.DataFrame(one_list, columns=['keyword', 'score'])
keyword_df

,keyword,score
0,room,0.4068
1,park,0.3957
2,booked,0.3467
3,garden,0.3106
4,place,0.3040
...,...,...
3851,bike,0.3555
3852,rent,0.3268
3853,location,0.3203
3854,staff,0.2639


In [78]:
# get the top 10 keywords by groupby keyword and get the sum of the score
keyword_df.groupby('keyword').sum().sort_values(by='score', ascending=False).head(10)




,score
keyword,
room,56.6953
staff,29.1907
bed,24.0215
location,16.9082
restaurant,16.5231
building,16.3890
bathroom,15.3067
breakfast,12.5277
park,10.8649


### may not be useful

In [79]:
import spacy
def remove_adj_adv(sentence):
    

    #load small english model from spacy: https://spacy.io/models/en
    nlp = spacy.load('en_core_web_sm') #you can use other methods
    # excluded tags
    excluded_tags = {"ADJ", "ADV"} #"NOUN", "VERB", "ADJ", "ADV", "ADP", "PROPN"

    new_sentence = []
    for token in nlp(sentence):
        if token.pos_ not in excluded_tags:
            new_sentence.append(token.text)
    new_sentence_str = " ".join(new_sentence)
    return new_sentence_str

#print and compare the original and the new sentence
print("Before:", test_df['Reviews_clean'][0])
print("After:", remove_adj_adv(test_df['Reviews_clean'][0]))
print()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_50571/2973746661.py:18 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_50571/2973746661.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'test_df' is not defined

In [80]:
#load small english model from spacy: https://spacy.io/models/en
nlp = spacy.load('en_core_web_sm') #you can use other methods
nlp(test_df['Reviews_clean'][0])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_50571/479142094.py:3 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/w2/5y09ynfn1dn9nzmzn5n1b4740000gp/T/ipykernel_50571/479142094.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'test_df' is not defined